# CAMEL Model for all the cooperatives

## Functions

In [1]:
import pandas as pd

### Capital Adequacy

In [2]:
# Loss of Assets
def loss_of_assets(excel_file, company_name, year, month):
    date_ = pd.Timestamp(f'{int(year)}-{int(month)}-01')
    
    mask1 = (excel_file['CUENTA'] == 300000) & (excel_file['fecha'] == date_)
    mask2 = (excel_file['CUENTA'] == 310000) & (excel_file['fecha'] == date_)
    
    account_coop1 = excel_file.loc[mask1, company_name]
    account_coop2 = excel_file.loc[mask2, company_name]
    
    if account_coop1.empty or account_coop2.empty:
        return 0

    value1 = list(account_coop1)[0]
    value2 = list(account_coop2)[0]

    return value1 / value2 if value2 != 0 else 0

# Solvency Ratio
def solvency_ratio():
    return 0

# Indicator of the relationship between the non-reducible minimum social contributions and the Share Capital
def min_social_contrib_social_capital(excel_file, company_name, year, month):
    date_ = pd.Timestamp(f'{int(year)}-{int(month)}-01')
    
    mask1 = (excel_file['CUENTA'] == 311000) & (excel_file['fecha'] == date_)
    mask2 = (excel_file['CUENTA'] == 310000) & (excel_file['fecha'] == date_)
    
    account_coop1 = excel_file.loc[mask1, company_name]
    account_coop2 = excel_file.loc[mask2, company_name]

    if account_coop1.empty or account_coop2.empty:
        return 0
    
    value1 = list(account_coop1)[0]
    value2 = list(account_coop2)[0]
    
    return value1 / value2 if value2 != 0 else 0


# --- Sum the values of every row 'i' in a specific company
def summation(excel_file, index_list, company_name, year, month):
    suma = 0
    date_ = pd.Timestamp(f'{int(year)}-{int(month)}-01')
    for i in index_list:
        mask = (excel_file['CUENTA'] == i) & (excel_file['fecha'] == date_)
        account_coop = excel_file.loc[mask, company_name]
        if account_coop.empty or account_coop.isna().all():
            value = 0
        else:
            value = account_coop.iloc[0]  # o .sum() si esperas múltiples filas
        suma += value
    return suma


# Indicator of the relationship between Institutional Capital and Total Assets
def capital_contribution_ratio(excel_file, company_name, year, month):
    date_ = pd.Timestamp(f'{int(year)}-{int(month)}-01')
    capital_inst_indices = [320000, 330000, 340000]
    capital_inst = summation(excel_file, capital_inst_indices, company_name, year, month)
    
    mask_total_assets = (excel_file['CUENTA'] == 100000) & (excel_file['fecha'] == date_)
    total_assets = excel_file.loc[mask_total_assets, company_name]
    if total_assets.empty:
        return 0
    total_assets_ = list(total_assets)
    value1 = capital_inst
    value2 = total_assets_[0]

    return value1 / value2 if value2 != 0 else 0

### Assets Quality

In [20]:
# Functions for quality indicator by risk with penalties
def gross_portfolio(excel_file, company_name, year, month):
    gross_portfolio_indices = [
        140400, 140500, 141100, 141200, 144100, 144200,
        144800, 145400, 145500, 146100, 146200, 146900
    ]
    return summation(excel_file, gross_portfolio_indices, company_name, year, month)

def risk_quality_indicator(excel_file, company_name, year, month):
    risk_portfolio_indices = [
        140410, 140415, 140420, 140425, 140510, 140515, 140520, 140525,
        141110, 141115, 141120, 141125, 141210, 141215, 141220, 141225,
        144110, 144115, 144120, 144125, 144210, 144215, 144220, 144225,
        144810, 144815, 144820, 144825, 145410, 145415, 145420, 145425,
        145510, 145515, 145520, 145525, 146110, 146115, 146120, 146125,
        146210, 146215, 146220, 146225, 146910, 146915, 146920, 146925,
        146935, 146940, 146945, 146950
    ]
    qualified_portfolio = summation(excel_file, risk_portfolio_indices, company_name, year, month)
    gross_portf = gross_portfolio(excel_file,company_name, year, month)

    return qualified_portfolio / gross_portf if gross_portf != 0 else 0

def risk_quality_with_writeoffs(excel_file, company_name, year, month):
    # total qualified portfolio
    qualified_portfolio = risk_quality_indicator(excel_file, company_name, year, month)
    # writeoffs
    mask = (excel_file['CUENTA'] == 831015) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01')
    writeoffs_list = list(excel_file.loc[mask, company_name])
    writeoffs = writeoffs_list[0] if writeoffs_list else 0
    total_with_writeoffs_indices = [
        140400, 140500, 141100, 141200, 144100, 144200, 144800,
        145400, 145500, 146100, 146200, 146900, 831015
    ]
    total_with_writeoffs = summation(excel_file, total_with_writeoffs_indices, company_name, year, month)

    return (qualified_portfolio + writeoffs) / total_with_writeoffs if total_with_writeoffs != 0 else 0

# Total Portfolio at Risk Coverage Indicator
def total_risk_coverage_indicator(excel_file, company_name, year, month):
    deterioration_indices = [140800, 144500, 145100, 145800, 146500, 146800, 147100]
    provisions = summation(excel_file, deterioration_indices, company_name, year, month)
    gross_portf = gross_portfolio(excel_file, company_name, year, month)
    return provisions / gross_portf if gross_portf != 0 else 0

# Productive asset
def productive_assets_ratio(excel_file, company_name, year, month):
    productive_assets_indices = [
        112000, 120000, 130000, 140405, 140410, 140505, 140510,
        141105, 141110, 141205, 141210, 144105, 144110, 144205,
        144210, 144805, 144810, 145405, 145410, 145505, 145510,
        146105, 146110, 146205, 146210, 146905, 146910, 146930,
        146935, 160505, 161505
    ]
    productive_assets = summation(excel_file, productive_assets_indices, company_name, year, month)
    mask = (excel_file['CUENTA'] == 100000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01')
    total_assets_list = list(excel_file.loc[mask, company_name])
    total_assets = total_assets_list[0] if total_assets_list else 0
    return productive_assets / total_assets if total_assets != 0 else 0

# Individual Coverage Indicator of the Unproductive Portfolio for the At-Risk Portfolio
def individual_coverage_nonproductive_portfolio(excel_file, company_name, year, month):
    provisions_cde_indices = [
        140815, 140820, 140825,
        144525, 144530, 144535, 144540, 144545, 144550,
        145115, 145120, 145125,
        145825, 145830, 145835, 145840, 145845, 145850,
        146525, 146530, 146535, 146540, 146545, 146550,
        147115, 147120, 147125, 147140, 147145, 147150
    ]
    provisions = summation(excel_file, provisions_cde_indices, company_name, year, month)

    overdue_portfolio_indices = [
        140415, 140420, 140425, 140515, 140520, 140525,
        141115, 141120, 141125, 141215, 141220, 141225,
        144115, 144120, 144125, 144215, 144220, 144225,
        144815, 144820, 144825, 145415, 145420, 145425,
        145515, 145520, 145525, 146115, 146120, 146125,
        146215, 146220, 146225, 146915, 146920, 146925,
        146940, 146945, 146950
    ]
    overdue = summation(excel_file, overdue_portfolio_indices, company_name, year, month)

    return provisions / overdue if overdue != 0 else 0

### Managerial Quality

In [27]:
# Operating Financial Margin Indicator
def financial_margin_operation(excel_file, company_name, year, month):
    date_filter = f'{int(year)}-{int(month)}-01'

    pos_list = list(excel_file.loc[(excel_file['CUENTA'] == 410000) & (excel_file['fecha'] == date_filter), company_name])
    positive_margin = pos_list[0] if pos_list else 0

    neg_list1 = list(excel_file.loc[(excel_file['CUENTA'] == 610000) & (excel_file['fecha'] == date_filter), company_name])
    negative_margin1 = neg_list1[0] if neg_list1 else 0

    serie = excel_file.loc[(excel_file['CUENTA'] == 700000) & (excel_file['fecha'] == date_filter), company_name]
    negative_margin2 = 0 if serie.empty else serie.iloc[0]

    sales_list = list(excel_file.loc[(excel_file['CUENTA'] == 410000) & (excel_file['fecha'] == date_filter), company_name])
    sales_income = sales_list[0] if sales_list else 0

    negative_margin = negative_margin1 + negative_margin2
    return (positive_margin - negative_margin) / sales_income if sales_income != 0 else 0


# Operating Margin Indicator (usa 'summation', no se modifica)
def operational_margin(excel_file, company_name, year, month):
    income_pos_indices = [410000, 422500]
    income_neg_indices = [610000, 700000, 510500, 510700, 511000, 511500, 540000]
    sales_indices = [410000, 422500]
    income_pos = summation(excel_file, income_pos_indices, company_name, year, month)
    income_neg = summation(excel_file, income_neg_indices, company_name, year, month)
    sales_income = summation(excel_file, sales_indices, company_name, year, month)
    return (income_pos - income_neg) / sales_income if sales_income != 0 else 0

# Indicator of the relationship between financial obligations and total liabilities
def financial_obligations_ratio(excel_file, company_name, year, month):
    date_filter = f'{int(year)}-{int(month)}-01'

    obligations_list = list(excel_file.loc[(excel_file['CUENTA'] == 230000) & (excel_file['fecha'] == date_filter), company_name])
    obligations = obligations_list[0] if obligations_list else 0

    total_liabilities_list = list(excel_file.loc[(excel_file['CUENTA'] == 200000) & (excel_file['fecha'] == date_filter), company_name])
    total_liabilities = total_liabilities_list[0] if total_liabilities_list else 0

    return obligations / total_liabilities if total_liabilities != 0 else 0

# Balance structure (usa 'summation', no se modifica)
def balance_structure(excel_file, company_name, year, month):
    productive_assets_indices = [
        112000, 120000, 130000, 140405, 140410, 140505, 140510,
        141105, 141110, 141205, 141210, 144105, 144110, 144205,
        144210, 144805, 144810, 145405, 145410, 145505, 145510,
        146105, 146110, 146205, 146210, 146905, 146910, 146930,
        146935, 160505, 161505
    ]
    interest_liabilities_indices = [210000, 230000]
    productive_assets = summation(excel_file, productive_assets_indices, company_name, year, month)
    interest_liabilities = summation(excel_file, interest_liabilities_indices, company_name, year, month)
    return productive_assets / interest_liabilities if interest_liabilities != 0 else 0


### Earnings Strength

In [36]:
# Return on equity indicator
def ROE(excel_file, company_name, year, month, n=1):
    fecha_actual = pd.Period(f"{year}-{month:02d}", freq="M")
    fecha_str = str(fecha_actual.to_timestamp().date())
    meses_previos = [str((fecha_actual - i).to_timestamp().date()) for i in range(1, 13)]

    num_values = excel_file.loc[
        (excel_file["CUENTA"] == 530000) & (excel_file["fecha"] == fecha_str),
        company_name
    ].values
    num = num_values[0] if len(num_values) > 0 else 0

    valores = excel_file.loc[
        (excel_file["CUENTA"] == 300000) & (excel_file["fecha"].isin(meses_previos)),
        company_name
    ].values

    if len(valores) == 0 or n == 0 or num == 0:
        return None

    prom_300000 = valores.mean()
    roe = (1 + ((num / prom_300000) / n))**12 - 1
    return roe


# Net margin indicator
def net_margin_indicator(excel_file, company_name, year, month):
    net_surplus_list = list(excel_file.loc[(excel_file['CUENTA'] == 530000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'), company_name])
    net_surplus = net_surplus_list[0] if net_surplus_list else 0
    incomes_indices = [410000, 422500]
    total_incomes = summation(excel_file, incomes_indices, company_name, year, month)
    return net_surplus / total_incomes if total_incomes != 0 else 0

# Indicator of return on invested capital
def ROIC(excel_file, company_name, year, month, n=1):
    fecha_actual = pd.Period(f"{year}-{month:02d}", freq="M")
    meses_previos = [str((fecha_actual - i).to_timestamp().date()) for i in range(1, 13)]

    # Numerator
    num_values = excel_file.loc[
        (excel_file["CUENTA"] == 530000) & (excel_file["fecha"] == str(fecha_actual.to_timestamp().date())),
        company_name
    ].values
    num = num_values[0] if len(num_values) > 0 else 0

    # Denominator
    cuentas = [210000, 230000, 300000]
    promedios = []
    for cuenta in cuentas:
        valores = excel_file.loc[
            (excel_file["CUENTA"] == cuenta) & (excel_file["fecha"].isin(meses_previos)),
            company_name
        ].values
        prom = valores.mean() if len(valores) > 0 else 0
        promedios.append(prom)

    denom = sum(promedios)

    if denom == 0 or n == 0 or num == 0:
        return None  # division by zero or no data

    roic = (1 + ((num / denom) / n))**12 - 1
    return roic

### Liquidicy Efficiency

In [6]:
# Liquidity Risk Indicator
def liquidity_risk():
    return 0

## Create the csv for all the cooperatives

In [7]:
coops = pd.read_excel("../resultados/Datos_2013_2025_cooperativas.xlsx")
coops

,CUENTA,NOMBRE CUENTA,COOPERATIVA DE LOS PROFESIONALES DE LA SALUD COASMEDAS,COOPERATIVA DE AHORRO Y CREDITO PARA EL BIENESTAR SOCIAL,COOPERATIVA PARA EL BIENESTAR SOCIAL,COOPERATIVA FINANCIERA SAN FRANCISCO,COOPERATIVA MULTIACTIVA DE LA AVIACION CIVIL COLOMBIANA,COOPERATIVA DE PROFESORES DE LA U NACIONAL DE COLOMBIA,CAJA COOPERATIVA CREDICOOP,COOPERATIVA DE AHORRO Y CREDITO DE SURAMERICA,...,COOPERATIVA DE AHORRO Y CREDITO COOMPARTIR,COOPERATIVA DE AHORRO Y CREDITO BERLIN,COOPERATIVA DE AHORRO Y CREDITO DE AIPE,COOPERATIVA DE AHORRO Y CREDITO DE SANTANDER LIMITADA,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA DE AHORRO Y CREDITO FINANCIAFONDOS,COOPERATIVA DE AHORRO Y CREDITO DE DROGUISTAS DETALLISTAS,COOPERATIVA DE AHORRO Y CREDITO COLANTA,COOPERATIVA ESPECIALIZADA DE AHORRO Y CREDITO AFROAMERICANA,fecha
0,100000,ACTIVO,3.387150e+11,1.560676e+11,9.879448e+10,1.064674e+10,7.021719e+10,1.417133e+11,8.491366e+10,2.198054e+10,...,4.312265e+09,4.443368e+10,2.850488e+10,1.641308e+12,3.832060e+11,1.559309e+10,2.597647e+11,3.174706e+11,7.498950e+09,2022-12-01
1,110000,EFECTIVO Y EQUIVALENTE AL EFECTIVO,1.566943e+10,2.669495e+10,1.918907e+09,1.552585e+09,4.685907e+09,1.434015e+10,6.147909e+09,2.294268e+09,...,1.192062e+08,2.639965e+09,2.223880e+09,2.443691e+11,7.403156e+09,2.817348e+08,3.586048e+10,2.764945e+10,5.507006e+08,2022-12-01
2,110500,CAJA,8.229352e+08,1.100000e+06,3.589941e+08,1.718631e+08,1.241549e+08,6.995092e+08,9.522290e+07,1.118834e+07,...,2.765270e+07,2.401334e+08,2.473210e+08,1.236956e+10,2.169476e+09,0.000000e+00,2.000000e+06,5.735745e+09,4.744467e+08,2022-12-01
3,110505,CAJA GENERAL,8.180852e+08,0.000000e+00,3.589941e+08,1.718631e+08,1.146549e+08,6.995092e+08,9.522290e+07,1.048834e+07,...,2.732270e+07,2.394334e+08,2.473210e+08,1.235737e+10,2.168001e+09,0.000000e+00,0.000000e+00,5.732195e+09,4.738467e+08,2022-12-01
4,110510,CAJA MENOR,4.850000e+06,1.100000e+06,0.000000e+00,0.000000e+00,9.500000e+06,0.000000e+00,0.000000e+00,7.000000e+05,...,3.300000e+05,7.000000e+05,0.000000e+00,1.218300e+07,1.475000e+06,0.000000e+00,2.000000e+06,3.550000e+06,6.000000e+05,2022-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292668,935000,OTRAS ACREEDORAS DE CONTROL,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.934962e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2014-12-01
292669,960000,ACREEDORAS CONTINGENTES POR CONTRA,2.645291e+11,1.493243e+08,5.438012e+10,2.747754e+09,4.058720e+10,2.363426e+11,9.935065e+10,1.808016e+10,...,8.768710e+09,4.080666e+10,9.539936e+09,1.535086e+12,1.871075e+11,9.630938e+09,2.930567e+11,2.023843e+11,6.737913e+09,2014-12-01
292670,960500,ACREEDORAS CONTINGENTES POR CONTRA (DB),2.645291e+11,1.493243e+08,5.438012e+10,2.747754e+09,4.058720e+10,2.363426e+11,9.935065e+10,1.808016e+10,...,8.768710e+09,4.080666e+10,9.539936e+09,1.535086e+12,1.871075e+11,9.630938e+09,2.930567e+11,2.023843e+11,6.737913e+09,2014-12-01
292671,980000,ACREEDORAS DE CONTROL POR CONTRA,2.619920e+10,3.159206e+11,7.700000e+09,1.299760e+09,6.160000e+09,0.000000e+00,1.232000e+10,3.080000e+09,...,1.274480e+09,3.654957e+09,1.386000e+09,8.014403e+10,8.129557e+09,1.289659e+09,5.000000e+09,2.140509e+10,2.266800e+09,2014-12-01


Generate new columns for the DataFrame

In [8]:
keys = list(coops.columns)
keys.remove("CUENTA")
keys.remove('NOMBRE CUENTA')
keys.remove('fecha')
print(keys)

['COOPERATIVA DE LOS PROFESIONALES DE LA SALUD COASMEDAS', 'COOPERATIVA DE AHORRO Y CREDITO PARA EL BIENESTAR SOCIAL', 'COOPERATIVA PARA EL BIENESTAR SOCIAL', 'COOPERATIVA FINANCIERA SAN FRANCISCO', 'COOPERATIVA MULTIACTIVA DE LA AVIACION CIVIL COLOMBIANA', 'COOPERATIVA DE PROFESORES DE LA U NACIONAL DE COLOMBIA', 'CAJA COOPERATIVA CREDICOOP', 'COOPERATIVA DE AHORRO Y CREDITO DE SURAMERICA', 'FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS', 'COOPERATIVA DEL MAGISTERIO', 'COOPERATIVA DE AHORRO Y CREDITO DE CHIPAQUE', 'COOPERATIVA DE AHORRO Y CREDITO UNIVERSIDAD SANTO TOMAS', 'CAJA COOPERATIVA PETROLERA', 'COOPERATIVA TEXAS LTDA', 'COOPERATIVA DE LOS TRABAJADORES DEL INSTITUTO DE LOS SEGUROS SOCIALES', 'COOPERATIVA DE TRABAJADORES DE BAVARIA DIRECCION Y VENTAS LTDA', 'COPROCENVA COOPERATIVA DE AHORRO Y CREDITO', 'COOPERATIVA DE AHORRO Y CREDITO DE TENJO', 'COOPERATIVA DE TRABAJADORES Y PENSIONADOS DE LA E.A.A.B.', 'COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA', 'COOPERATIVA MULTIA

In [9]:
indexes_camel = ['Quebranto Patrimonial', 'Relación Solvencia', 'Relación entre Aportes sociales mínimos no reducibles y Capital Social', 'Relación entre el Capital Institucional y el Activo Total', 
                 'Indicador de calidad por riesgo', 'Indicador de calidad por riesgo con castigos', 'Indicador de Cobertura de la Cartera Total en Riesgo', 'Activo Productivo', 'Indicador de Cobertura individual de la cartera improductiva para la cartera en Riesgo',
                 'Indicador de Margen Financiero de Operación', 'Indicador de Margen Operacional', 'Indicador de relación entre las obligaciones financieras y el pasivo total', 'Estructura de Balance',
                 'Indicador de rentabilidad sobre recursos propios - ROE', 'Indicador de margen neto', 'Indicador de rentabilidad sobre el capital invertido - ROIC',
                 'Indicador de Riesgo de Liquidez - IRL']
years = range(2014, 2026)
months = range(1, 13)

We need to parallelize the next cells, because there is a lot of calculations there (for the big dataset)

### Capital

In [10]:
from concurrent.futures import ProcessPoolExecutor
from functools import partial

In [11]:
def calculate_values_capital(company, coops, years, months, indices_camel):
    rows = []
    for y in years:
        for m in months:
            c1 = loss_of_assets(coops, company, y, m)
            c2 = solvency_ratio()
            c3 = min_social_contrib_social_capital(coops, company, y, m)
            c4 = capital_contribution_ratio(coops, company, y, m)

            rows.append([y, m, indices_camel[0], company, c1])
            rows.append([y, m, indices_camel[1], company, c2])
            rows.append([y, m, indices_camel[2], company, c3])
            rows.append([y, m, indices_camel[3], company, c4])
    return rows

# parameters
partial_func_c = partial(calculate_values_capital, coops=coops, years=years, months=months, indices_camel=indexes_camel)

# execute for every company
with ProcessPoolExecutor() as executor:
    resultados = executor.map(partial_func_c, keys)

# organize all the info in a dataframe
rows_c = [fila for sublista in resultados for fila in sublista]

df = pd.DataFrame(rows_c, columns=['Año', 'Mes', 'índice CAMEL', 'Cooperativa', 'valor CAMEL'])
df_pivot = df.pivot_table(index=['Año', 'Mes', 'índice CAMEL'], columns='Cooperativa', values='valor CAMEL').reset_index()
df_pivot.columns.name = None

In [12]:
df_pivot

,Año,Mes,índice CAMEL,CAJA COOPERATIVA CREDICOOP,CAJA COOPERATIVA PETROLERA,CASA NACIONAL DEL PROFESOR,COOPANTEX COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA AHORRO Y CREDITO GOMEZ PLATA LTDA.,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA CALDENSE DEL PROFESOR,...,COOPERATIVA SAN VICENTE DE PAUL LTDA.,COOPERATIVA TELEPOSTAL LTDA,COOPERATIVA TEXAS LTDA,COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA,COPERATIVA MULTIACTIVA DE EDUCADORES DE BOYACA,COPROCENVA COOPERATIVA DE AHORRO Y CREDITO,EMPRESA COOPERATIVA DE AHORRO Y CREDITO SIGLO XX LTDA.,FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS,GRAN COOPERATIVA DE ENERGIA ELECTRICA Y RECURSOS NATURALES,"MULTIACTIVA EL ROBLE, ENTIDAD COOPERATIVA"
0,2014,1,Quebranto Patrimonial,1.500449,1.310736,1.188087,1.588034,1.481415,2.088982,0.000000,...,1.366898,1.575925,1.430989,1.415437,1.119304,1.367164,1.328639,1.615208,1.634529,2.159444
1,2014,1,Relación Solvencia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2014,1,Relación entre Aportes sociales mínimos no red...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2014,1,Relación entre el Capital Institucional y el A...,0.145260,0.057027,0.047077,0.126778,0.085107,0.094572,0.000000,...,0.072971,0.157531,0.112465,0.066512,0.031398,0.060938,0.152577,0.116553,0.243769,0.256789
4,2014,2,Quebranto Patrimonial,1.508398,1.310754,1.190007,1.597500,1.480301,2.099399,1.001952,...,1.367030,1.581733,1.440119,1.415271,1.117340,1.370708,1.337403,1.625810,1.637438,2.173214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,2025,11,Relación entre el Capital Institucional y el A...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
572,2025,12,Quebranto Patrimonial,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
573,2025,12,Relación Solvencia,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
574,2025,12,Relación entre Aportes sociales mínimos no red...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Assets

In [13]:
def calculate_values_assets_1(company, coops, years, months, indices_camel):
    rows = []
    for y in years:
        for m in months:
            a1 = risk_quality_indicator(coops, company, y, m)
            a2 = risk_quality_with_writeoffs(coops, company, y, m)
            a3 = total_risk_coverage_indicator(coops, company, y, m)

            rows.append([y, m, indices_camel[4], company, a1])
            rows.append([y, m, indices_camel[5], company, a2])
            rows.append([y, m, indices_camel[6], company, a3])
    return rows


def calculate_values_assets_2(company, coops, years, months, indices_camel):
    rows = []
    for y in years:
        for m in months:
            # values for A: ASSETS
            a4 = productive_assets_ratio(coops, company, y, m)
            a5 = individual_coverage_nonproductive_portfolio(coops, company, y, m)

            rows.append([y, m, indices_camel[7], company, a4])
            rows.append([y, m, indices_camel[8], company, a5])
    return rows      

first part

In [ ]:
partial_func_a = partial(calculate_values_assets_1, coops=coops, years=years, months=months, indices_camel=indexes_camel)

# every company
with ProcessPoolExecutor(max_workers=4) as executor:
    resultados = list(executor.map(partial_func_a, keys))

rows_a = [fila for sublista in resultados for fila in sublista]

# Filtra filas donde el valor CAMEL no es nulo/NaN y el año es 2025
rows_a_filtrados = [fila for fila in rows_a if fila[0] == 2025 and pd.notnull(fila[4]) and fila[4] != 0]

# incluir todos los años pero omitir cooperativas sin valores en 2025:
# Primero identifica las cooperativas válidas
coops_con_valores_2025 = set(fila[3] for fila in rows_a if fila[0] == 2025 and pd.notnull(fila[4]) and fila[4] != 0)
rows_a_filtrados = [fila for fila in rows_a if fila[3] in coops_con_valores_2025]

df_a = pd.DataFrame(rows_a_filtrados, columns=['Año', 'Mes', 'índice CAMEL', 'Cooperativa', 'valor CAMEL'])
df_pivot_a = df_a.pivot_table(index=['Año', 'Mes', 'índice CAMEL'], columns='Cooperativa', values='valor CAMEL').reset_index()
df_pivot_a.columns.name = None


In [18]:
df_pivot_a

,Año,Mes,índice CAMEL,CAJA COOPERATIVA CREDICOOP,CAJA COOPERATIVA PETROLERA,CASA NACIONAL DEL PROFESOR,COOPANTEX COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA AHORRO Y CREDITO GOMEZ PLATA LTDA.,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA CALDENSE DEL PROFESOR,...,COOPERATIVA SAN VICENTE DE PAUL LTDA.,COOPERATIVA TELEPOSTAL LTDA,COOPERATIVA TEXAS LTDA,COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA,COPERATIVA MULTIACTIVA DE EDUCADORES DE BOYACA,COPROCENVA COOPERATIVA DE AHORRO Y CREDITO,EMPRESA COOPERATIVA DE AHORRO Y CREDITO SIGLO XX LTDA.,FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS,GRAN COOPERATIVA DE ENERGIA ELECTRICA Y RECURSOS NATURALES,"MULTIACTIVA EL ROBLE, ENTIDAD COOPERATIVA"
0,2014,1,Indicador de Cobertura de la Cartera Total en ...,0.005839,0.000417,0.000000,0.017426,0.000000,0.004162,0.000000,...,0.000000,0.000000,0.000000,0.048988,0.000000,0.073818,0.000000,0.104007,0.000000,0.000094
1,2014,1,Indicador de calidad por riesgo,0.108764,0.110324,0.031787,0.078034,0.059702,0.052288,0.000000,...,0.065341,0.026896,0.017434,0.060727,0.041846,0.019537,0.078490,0.051485,0.023378,0.019847
2,2014,1,Indicador de calidad por riesgo con castigos,0.011053,0.038309,0.003502,0.039453,0.037028,0.036175,0.000000,...,0.101027,0.006853,0.001073,0.045772,0.000784,0.008800,0.072061,0.085476,0.008264,0.003875
3,2014,2,Indicador de Cobertura de la Cartera Total en ...,0.015328,0.000445,0.000000,0.017244,0.000000,0.004461,0.000000,...,0.000000,0.000000,0.000000,0.078588,0.000000,0.073568,0.000000,0.104858,0.000000,0.000088
4,2014,2,Indicador de calidad por riesgo,0.108134,0.116331,0.031768,0.076533,0.054753,0.055550,0.104073,...,0.061050,0.029571,0.012309,0.061186,0.036046,0.018609,0.075717,0.059812,0.026135,0.019944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,2025,11,Indicador de calidad por riesgo,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
428,2025,11,Indicador de calidad por riesgo con castigos,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
429,2025,12,Indicador de Cobertura de la Cartera Total en ...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
430,2025,12,Indicador de calidad por riesgo,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


second part

In [21]:
partial_func_a_2 = partial(calculate_values_assets_2, coops=coops, years=years, months=months, indices_camel=indexes_camel)

# every company
with ProcessPoolExecutor(max_workers=4) as executor:
    resultados_2 = list(executor.map(partial_func_a_2, keys))

rows_a_2 = [fila for sublista in resultados_2 for fila in sublista]

# incluir todos los años pero omitir cooperativas sin valores en 2025:
# Primero identifica las cooperativas válidas
coops_con_valores_2025_2 = set(fila[3] for fila in rows_a_2 if fila[0] == 2025 and pd.notnull(fila[4]) and fila[4] != 0)
rows_a_filtrados_2 = [fila for fila in rows_a_2 if fila[3] in coops_con_valores_2025_2]

df_a_2 = pd.DataFrame(rows_a_filtrados_2, columns=['Año', 'Mes', 'índice CAMEL', 'Cooperativa', 'valor CAMEL'])
df_pivot_a_2 = df_a_2.pivot_table(index=['Año', 'Mes', 'índice CAMEL'], columns='Cooperativa', values='valor CAMEL').reset_index()
df_pivot_a_2.columns.name = None

In [22]:
df_pivot_a_2

,Año,Mes,índice CAMEL,CAJA COOPERATIVA CREDICOOP,CAJA COOPERATIVA PETROLERA,CASA NACIONAL DEL PROFESOR,COOPANTEX COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA AHORRO Y CREDITO GOMEZ PLATA LTDA.,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA CALDENSE DEL PROFESOR,...,COOPERATIVA SAN VICENTE DE PAUL LTDA.,COOPERATIVA TELEPOSTAL LTDA,COOPERATIVA TEXAS LTDA,COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA,COPERATIVA MULTIACTIVA DE EDUCADORES DE BOYACA,COPROCENVA COOPERATIVA DE AHORRO Y CREDITO,EMPRESA COOPERATIVA DE AHORRO Y CREDITO SIGLO XX LTDA.,FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS,GRAN COOPERATIVA DE ENERGIA ELECTRICA Y RECURSOS NATURALES,"MULTIACTIVA EL ROBLE, ENTIDAD COOPERATIVA"
0,2014,1,Activo Productivo,0.852824,0.851907,0.785281,0.811272,0.784887,0.921653,0.000000,...,0.939198,0.949141,0.923584,0.794342,0.839536,0.835262,0.805171,0.766407,0.964487,0.906757
1,2014,1,Indicador de Cobertura individual de la carter...,0.000000,0.000445,0.000000,0.061149,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.060056,0.000000,0.197307,0.000000,0.000000
2,2014,2,Activo Productivo,0.850575,0.855011,0.784087,0.810077,0.809313,0.920249,0.666561,...,0.941147,0.957601,0.930651,0.784443,0.845968,0.845815,0.807043,0.748662,0.976286,0.902241
3,2014,2,Indicador de Cobertura individual de la carter...,0.000000,0.000396,0.000000,0.061854,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.066661,0.000000,0.194980,0.000000,0.000000
4,2014,3,Activo Productivo,0.860853,0.854762,0.801462,0.809778,0.807407,0.924471,0.654243,...,0.951586,0.000000,0.937075,0.783777,0.850747,0.844396,0.818895,0.738710,0.980189,0.895490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2025,10,Indicador de Cobertura individual de la carter...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
284,2025,11,Activo Productivo,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
285,2025,11,Indicador de Cobertura individual de la carter...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
286,2025,12,Activo Productivo,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Managerial

In [23]:
def calculate_values_managerial(company, coops, years, months, indices_camel):
    rows = []
    for y in years:
        for m in months:
            # values for M: MANAGERIAL
            m1 = financial_margin_operation(coops, company, y, m)
            m2 = operational_margin(coops, company, y, m)
            m3 = financial_obligations_ratio(coops, company, y, m)
            m4 = balance_structure(coops, company, y, m)

            rows.append([y, m, indices_camel[9], company, m1])
            rows.append([y, m, indices_camel[10], company, m2])
            rows.append([y, m, indices_camel[11], company, m3])
            rows.append([y, m, indices_camel[12], company, m4])
    return rows


In [28]:
partial_func_m = partial(calculate_values_managerial, coops=coops, years=years, months=months, indices_camel=indexes_camel)

# every company
with ProcessPoolExecutor(max_workers=4) as executor:
    resultados = list(executor.map(partial_func_m, keys))

rows_m = [fila for sublista in resultados for fila in sublista]

df_m = pd.DataFrame(rows_m, columns=['Año', 'Mes', 'índice CAMEL', 'Cooperativa', 'valor CAMEL'])
df_pivot_m = df_m.pivot_table(index=['Año', 'Mes', 'índice CAMEL'], columns='Cooperativa', values='valor CAMEL').reset_index()
df_pivot_m.columns.name = None

In [29]:
df_pivot_m

,Año,Mes,índice CAMEL,CAJA COOPERATIVA CREDICOOP,CAJA COOPERATIVA PETROLERA,CASA NACIONAL DEL PROFESOR,COOPANTEX COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA AHORRO Y CREDITO GOMEZ PLATA LTDA.,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA CALDENSE DEL PROFESOR,...,COOPERATIVA SAN VICENTE DE PAUL LTDA.,COOPERATIVA TELEPOSTAL LTDA,COOPERATIVA TEXAS LTDA,COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA,COPERATIVA MULTIACTIVA DE EDUCADORES DE BOYACA,COPROCENVA COOPERATIVA DE AHORRO Y CREDITO,EMPRESA COOPERATIVA DE AHORRO Y CREDITO SIGLO XX LTDA.,FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS,GRAN COOPERATIVA DE ENERGIA ELECTRICA Y RECURSOS NATURALES,"MULTIACTIVA EL ROBLE, ENTIDAD COOPERATIVA"
0,2014,1,Estructura de Balance,2.001510,1.386005,1.323835,1.286730,1.207956,1.211381,0.000000,...,1.511317,2.828472,1.873193,1.127042,1.267051,1.264017,2.452405,1.352501,4.447264,1.918250
1,2014,1,Indicador de Margen Financiero de Operación,0.854999,0.766067,0.735190,0.662201,0.793170,0.776994,0.000000,...,0.767412,0.893440,0.779820,0.703994,0.719127,0.786126,0.858041,0.789625,0.932666,0.842562
2,2014,1,Indicador de Margen Operacional,-0.024100,0.172809,0.325298,0.069617,0.245429,0.111092,0.000000,...,0.171244,0.310847,0.325096,0.159394,0.153694,0.273928,0.042445,0.138058,0.450585,0.126589
3,2014,1,Indicador de relación entre las obligaciones f...,0.124309,0.248687,0.006845,0.074817,0.000000,0.000000,0.000000,...,0.145249,0.000000,0.000744,0.012726,0.012376,0.052229,0.240873,0.005650,0.004337,0.000000
4,2014,2,Estructura de Balance,2.030194,1.398220,1.326969,1.298057,1.248806,1.201699,1.823263,...,1.492343,2.899332,1.909459,1.115010,1.277784,1.273710,2.501366,1.339850,4.454662,1.889526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,2025,11,Indicador de relación entre las obligaciones f...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
572,2025,12,Estructura de Balance,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
573,2025,12,Indicador de Margen Financiero de Operación,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
574,2025,12,Indicador de Margen Operacional,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Earnings

In [30]:
def calculate_values_earnings(company, coops, years, months, indices_camel):
    rows = []
    for y in years:
        for m in months:
            # values for E: EARNINGS
            e1 = ROE(coops, company, y, m)
            e2 = net_margin_indicator(coops, company, y, m)
            e3 = ROIC(coops, company, y, m)

            rows.append([y, m, indices_camel[13], company, e1])
            rows.append([y, m, indices_camel[14], company, e2])
            rows.append([y, m, indices_camel[15], company, e3])
    return rows

In [37]:
partial_func_e = partial(calculate_values_earnings, coops=coops, years=years, months=months, indices_camel=indexes_camel)

# every company
with ProcessPoolExecutor(max_workers=4) as executor:
    resultados = list(executor.map(partial_func_e, keys))

rows_e = [fila for sublista in resultados for fila in sublista]

df_e = pd.DataFrame(rows_e, columns=['Año', 'Mes', 'índice CAMEL', 'Cooperativa', 'valor CAMEL'])
df_pivot_e = df_e.pivot_table(index=['Año', 'Mes', 'índice CAMEL'], columns='Cooperativa', values='valor CAMEL').reset_index()
df_pivot_e.columns.name = None

In [38]:
df_pivot_e

,Año,Mes,índice CAMEL,CAJA COOPERATIVA CREDICOOP,CAJA COOPERATIVA PETROLERA,CASA NACIONAL DEL PROFESOR,COOPANTEX COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA AHORRO Y CREDITO GOMEZ PLATA LTDA.,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA CALDENSE DEL PROFESOR,...,COOPERATIVA SAN VICENTE DE PAUL LTDA.,COOPERATIVA TELEPOSTAL LTDA,COOPERATIVA TEXAS LTDA,COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA,COPERATIVA MULTIACTIVA DE EDUCADORES DE BOYACA,COPROCENVA COOPERATIVA DE AHORRO Y CREDITO,EMPRESA COOPERATIVA DE AHORRO Y CREDITO SIGLO XX LTDA.,FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS,GRAN COOPERATIVA DE ENERGIA ELECTRICA Y RECURSOS NATURALES,"MULTIACTIVA EL ROBLE, ENTIDAD COOPERATIVA"
0,2014,1,Indicador de margen neto,0.103550,0.006416,0.101504,0.008841,0.078828,0.000000,0.0,...,0.022719,0.0,0.000602,0.016322,0.002736,0.132087,0.010363,0.022700,0.096347,0.016085
1,2014,1,Indicador de rentabilidad sobre el capital inv...,0.030023,0.001522,0.019363,0.001622,0.287629,NaN,NaN,...,0.006100,NaN,0.000095,0.003831,0.000511,0.026929,0.027896,0.005241,0.017422,0.002764
2,2014,1,Indicador de rentabilidad sobre recursos propi...,0.060634,0.004376,0.051102,0.004212,1.091621,NaN,NaN,...,0.018233,NaN,0.000201,0.014137,0.001256,0.085972,0.041858,0.012665,0.022384,0.005003
3,2014,2,Indicador de margen neto,0.056691,0.005839,0.101388,0.008810,0.079381,0.007567,0.0,...,0.018992,0.0,0.000846,0.021638,0.002719,0.133724,0.004142,0.025882,0.105009,0.016541
4,2014,2,Indicador de rentabilidad sobre el capital inv...,0.031084,0.002772,0.038031,0.003231,0.310577,0.003338,NaN,...,0.010734,NaN,0.000266,0.010094,0.001009,0.054434,0.000916,0.012049,0.035054,0.005619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,2025,8,Indicador de margen neto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
408,2025,9,Indicador de margen neto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
409,2025,10,Indicador de margen neto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
410,2025,11,Indicador de margen neto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Liquidicy

In [39]:
def calculate_values_liquidicy(company, coops, years, months, indices_camel):
    rows = []
    for y in years:
        for m in months:
            # values for L: LIQUIDICY
            l1 = liquidity_risk()
            rows.append([y, m, indices_camel[16], company, l1])
    return rows

In [40]:
partial_func_l = partial(calculate_values_liquidicy, coops=coops, years=years, months=months, indices_camel=indexes_camel)

# every company
with ProcessPoolExecutor(max_workers=4) as executor:
    resultados = list(executor.map(partial_func_l, keys))

rows_l = [fila for sublista in resultados for fila in sublista]

df_l = pd.DataFrame(rows_l, columns=['Año', 'Mes', 'índice CAMEL', 'Cooperativa', 'valor CAMEL'])
df_pivot_l = df_l.pivot_table(index=['Año', 'Mes', 'índice CAMEL'], columns='Cooperativa', values='valor CAMEL').reset_index()
df_pivot_l.columns.name = None

In [41]:
df_pivot_l

,Año,Mes,índice CAMEL,CAJA COOPERATIVA CREDICOOP,CAJA COOPERATIVA PETROLERA,CASA NACIONAL DEL PROFESOR,COOPANTEX COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA AHORRO Y CREDITO GOMEZ PLATA LTDA.,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA CALDENSE DEL PROFESOR,...,COOPERATIVA SAN VICENTE DE PAUL LTDA.,COOPERATIVA TELEPOSTAL LTDA,COOPERATIVA TEXAS LTDA,COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA,COPERATIVA MULTIACTIVA DE EDUCADORES DE BOYACA,COPROCENVA COOPERATIVA DE AHORRO Y CREDITO,EMPRESA COOPERATIVA DE AHORRO Y CREDITO SIGLO XX LTDA.,FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS,GRAN COOPERATIVA DE ENERGIA ELECTRICA Y RECURSOS NATURALES,"MULTIACTIVA EL ROBLE, ENTIDAD COOPERATIVA"
0,2014,1,Indicador de Riesgo de Liquidez - IRL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014,2,Indicador de Riesgo de Liquidez - IRL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014,3,Indicador de Riesgo de Liquidez - IRL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014,4,Indicador de Riesgo de Liquidez - IRL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2014,5,Indicador de Riesgo de Liquidez - IRL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2025,8,Indicador de Riesgo de Liquidez - IRL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140,2025,9,Indicador de Riesgo de Liquidez - IRL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141,2025,10,Indicador de Riesgo de Liquidez - IRL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142,2025,11,Indicador de Riesgo de Liquidez - IRL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## CSV for BD

Write on .csv the data from DataFrames:

DataFrames's names and their title:
* 'df_pivot': Capital
* 'df_pivot_a' y 'df_pivot_a_2': Assets
* 'df_pivot_m': Managerial
* 'df_pivot_e': Earnings
* 'df_pivot_l': Liquidicy

In [42]:
df_pivot.to_csv("Capital.csv", index=False, encoding="utf-8")
# assets
assets = pd.concat([df_pivot_a, df_pivot_a_2], ignore_index=True)
assets.to_csv("Assets.csv", index=False, encoding="utf-8")
# ---
df_pivot_m.to_csv("Managerial.csv", index=False, encoding="utf-8")
df_pivot_e.to_csv("Earnings.csv", index=False, encoding="utf-8")
df_pivot_l.to_csv("Liquidity.csv", index=False, encoding="utf-8")